In [34]:
import pandas as pd
import numpy as np
import math

In [25]:
gamesdf = pd.read_csv("../resources/clean_games.csv")
gamesdf = gamesdf.drop(gamesdf[["Unnamed: 0"]], axis=1)

gamesdf

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2022-12-22,22200477,1610612740,1610612759,2022,1610612740,126.0,0.484,0.926,0.382,25.0,46.0,1610612759,117.0,0.478,0.815,0.321,23.0,44.0,1
1,2022-12-22,22200478,1610612762,1610612764,2022,1610612762,120.0,0.488,0.952,0.457,16.0,40.0,1610612764,112.0,0.561,0.765,0.333,20.0,37.0,1
2,2022-12-21,22200466,1610612739,1610612749,2022,1610612739,114.0,0.482,0.786,0.313,22.0,37.0,1610612749,106.0,0.470,0.682,0.433,20.0,46.0,1
3,2022-12-21,22200467,1610612755,1610612765,2022,1610612755,113.0,0.441,0.909,0.297,27.0,49.0,1610612765,93.0,0.392,0.735,0.261,15.0,46.0,1
4,2022-12-21,22200468,1610612737,1610612741,2022,1610612737,108.0,0.429,1.000,0.378,22.0,47.0,1610612741,110.0,0.500,0.773,0.292,20.0,47.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26547,2014-10-06,11400007,1610612737,1610612740,2014,1610612737,93.0,0.419,0.821,0.421,24.0,50.0,1610612740,87.0,0.366,0.643,0.375,17.0,43.0,1
26548,2014-10-06,11400004,1610612741,1610612764,2014,1610612741,81.0,0.338,0.719,0.381,18.0,40.0,1610612764,85.0,0.411,0.636,0.267,17.0,47.0,0
26549,2014-10-06,11400005,1610612747,1610612743,2014,1610612747,98.0,0.448,0.682,0.500,29.0,45.0,1610612743,95.0,0.387,0.659,0.500,19.0,43.0,1
26550,2014-10-05,11400002,1610612761,1610612758,2014,1610612761,99.0,0.440,0.771,0.333,21.0,30.0,1610612758,94.0,0.469,0.725,0.385,18.0,45.0,1


In [43]:
detailsdf = pd.read_csv("../resources/clean_games_details.csv")
detailsdf[["MIN", "SECOND"]] = detailsdf["MIN"].str.split(":", expand=True)
detailsdf = detailsdf.drop(detailsdf[["Unnamed: 0", "SECOND"]], axis=1)
detailsdf =  detailsdf.dropna()
detailsdf.isnull().sum()

/Users/mitch/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


GAME_ID        0
TEAM_ID        0
PLAYER_ID      0
PLAYER_NAME    0
MIN            0
FGM            0
FGA            0
FG_PCT         0
FG3M           0
FG3A           0
FG3_PCT        0
FTM            0
FTA            0
FT_PCT         0
REB            0
AST            0
STL            0
BLK            0
TO             0
PTS            0
dtype: int64

In [44]:
detailsdf

,GAME_ID,TEAM_ID,PLAYER_ID,PLAYER_NAME,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,REB,AST,STL,BLK,TO,PTS
0,22200477,1610612759,1629641,Romeo Langford,18,1.0,1.0,1.000,0.0,0.0,0.000,0.0,0.0,0.000,2.0,0.0,1.0,0.0,2.0,2.0
1,22200477,1610612759,1631110,Jeremy Sochan,31,7.0,14.0,0.500,2.0,4.0,0.500,7.0,10.0,0.700,9.0,6.0,1.0,0.0,2.0,23.0
2,22200477,1610612759,1627751,Jakob Poeltl,21,6.0,9.0,0.667,0.0,0.0,0.000,1.0,1.0,1.000,4.0,1.0,1.0,0.0,2.0,13.0
3,22200477,1610612759,1630170,Devin Vassell,30,4.0,13.0,0.308,1.0,6.0,0.167,1.0,1.0,1.000,9.0,5.0,3.0,0.0,2.0,10.0
4,22200477,1610612759,1630200,Tre Jones,27,7.0,12.0,0.583,1.0,3.0,0.333,4.0,4.0,1.000,2.0,3.0,0.0,0.0,2.0,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
557051,11200086,1610612747,2768,Chris Duhon,8,1.0,3.0,0.333,0.0,1.0,0.000,0.0,0.0,0.000,1.0,2.0,1.0,0.0,0.0,2.0
557052,11200086,1610612747,202365,Devin Ebanks,15,2.0,5.0,0.400,0.0,2.0,0.000,0.0,0.0,0.000,1.0,1.0,0.0,0.0,1.0,4.0
557053,11200086,1610612747,2730,Dwight Howard,33,8.0,12.0,0.667,0.0,0.0,0.000,3.0,8.0,0.375,12.0,2.0,0.0,4.0,5.0,19.0
557054,11200086,1610612747,201975,Jodie Meeks,15,2.0,4.0,0.500,2.0,2.0,1.000,0.0,0.0,0.000,1.0,1.0,0.0,0.0,0.0,6.0


In [22]:
rankingdf = pd.read_csv("../resources/clean_ranking.csv")
rankingdf = rankingdf.drop(rankingdf[["Unnamed: 0"]], axis=1)
rankingdf.head()

,TEAM_ID,CONFERENCE,TEAM,G,W,L,W_PCT,DATE
0,1610612743,West,Denver,30,19,11,0.633,2022-12-22
1,1610612763,West,Memphis,30,19,11,0.633,2022-12-22
2,1610612740,West,New Orleans,31,19,12,0.613,2022-12-22
3,1610612756,West,Phoenix,32,19,13,0.594,2022-12-22
4,1610612746,West,LA Clippers,33,19,14,0.576,2022-12-22


In [9]:
playersdf = pd.read_csv("../resources/players.csv")
playersdf.head()

,PLAYER_NAME,TEAM_ID,PLAYER_ID,SEASON
0,Royce O'Neale,1610612762,1626220,2019
1,Bojan Bogdanovic,1610612762,202711,2019
2,Rudy Gobert,1610612762,203497,2019
3,Donovan Mitchell,1610612762,1628378,2019
4,Mike Conley,1610612762,201144,2019


In [10]:
gamesdf.to_csv("../resources/games.csv", index=False)
detailsdf.to_csv("../resources/game_details.csv", index=False)
rankingdf.to_csv("../resources/ranking.csv", index=False)
playersdf.to_csv("../resources/players.csv", index=False)